## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [25]:
import pandas as pd
import numpy as np
import sqlite3
import json

# Initial data investigation

In [5]:
cd zippedData

[Errno 2] No such file or directory: 'zippedData'
/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/zippedData


In [6]:
ls

bom.movie_gross.csv.gz        imdb.title.principals.csv.gz
column_summary.csv            imdb.title.ratings.csv.gz
imdb.name.basics.csv.gz       rt.movie_info.tsv.gz
imdb.title.akas.csv.gz        rt.reviews.tsv.gz
imdb.title.basics.csv.gz      tmdb.movies.csv.gz
imdb.title.crew.csv.gz        tn.movie_budgets.csv.gz


In [7]:
def preview_csvtsv(filepath, row_count):
    """Create a dataframe of initial n rows from file as a preview.
    Supports either csv or tsv file formats, and files compressed as gz.
    File format and gz detected using file extensions.
    Prints a preview dataframe of the file, the preview's head(), 
    and the total number of rows.
    """
    import gzip
    is_gz = False
    
    print(f"""--------------------------------------------------
    File: {filepath}
    --------------------------------------------------""")
    
    # determine file extension(s)
    if filepath.split('.')[-1].lower() == 'csv':
        filetype='csv'
    elif filepath.split('.')[-1].lower() == 'tsv':
        filetype='tsv'
    elif filepath.split('.')[-1].lower() == 'gz':
        # if we have a gz file, get second to last element
        is_gz = True
        if filepath.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif filepath.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
    else:
        # default to CSV if can't determine the file extention
        filetype='csv'
    
    # Process file based on extension. 
    # Pandas doesn't care if the file is a gz compressed
    if filetype == 'csv':
        df_test = pd.read_csv(filepath, nrows=row_count)
    elif filetype == 'tsv':
        df_test = pd.read_table(filepath, nrows=row_count)

    num_lines = 0
    
    # get line count. This differs based on if the file is gz or not
    if is_gz:
        for line in gzip.open(filepath):
            num_lines += 1
    else:
        for line in open(filepath):
            num_lines += 1
    
    display(df_test.head())
    display(df_test.info())
    print(f"\nFile contains {num_lines} total lines, including header.")
    print('\n\n\n')
    
    pass

In [8]:
def filelist_to_dfs_prelim(filepath_list, dfname_list):
    """Creates a dataframe for each file in the list.
    Names each dataframe using the corresponding name list.
    """
    df_dict = {}
    filetype = ''

    if len(filepath_list) != len(dfname_list):
        return "Error: lists must be of equivalent length."
    
    # loop through each file in the filepath_list
    for i, file in enumerate(filepath_list):
        
        print(f"Working on {file}...")
        
        # figure out the file type to use the appropriate method
        if file.split('.')[-1].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-1].lower() == 'tsv':
            filetype='tsv'
        elif file.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
        else:
            # default to CSV if can't determine the file extention
            filetype='csv'
        
        try:
            # determine file extension and apply appropriate method to load preview
            if filetype == 'tsv':
                df_dict[dfname_list[i]] = pd.read_table(file)
            else:
                # default to CSV if not tsv
                df_dict[dfname_list[i]] = pd.read_csv(file) 
        except:
            print(f"Error encountered and {file} was skipped")

    return df_dict
    

In [9]:
def filelist_to_dfs_errorh(filepath_list, dfname_list):
    """Creates a dataframe for each file in the list.
    Names each dataframe using the corresponding name list.
    """
    df_dict = {}
    filetype = ''
    
    if len(filepath_list) != len(dfname_list):
        return "Error: lists must be of equivalent length."
    
    # loop through each file in the filepath_list
    for i, file in enumerate(filepath_list):
        
        print(f"Working on {file}...")

        # figure out the file type to use the appropriate method
        if file.split('.')[-1].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-1].lower() == 'tsv':
            filetype='tsv'
        elif file.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
        else:
            # default to CSV if can't determine the file extention
            filetype='csv'
        
        # determine file extension and apply appropriate method to load df
        # Credit to https://stackoverflow.com/questions/33440805
        #     /pandas-dataframe-read-csv-on-bad-data for handling error rows
        if filetype == 'tsv':
            df_dict[dfname_list[i]] = pd.read_table(file
                                        , error_bad_lines = False
                                        , warn_bad_lines = True
                                        , encoding='latin1') 
        else:
            # default to CSV if not tsv
            df_dict[dfname_list[i]] = pd.read_csv(file
                                        , error_bad_lines = False
                                        , warn_bad_lines = True
                                        , encoding='latin1') 

    return df_dict
    

In [10]:
def summarize_cols(df_dict):
    """Creates a dataframe with df/column summary info.
    Takes a dictionary as input, where the key is the name of the df,
    and the value is the dataframe itself."""
    
    arr_colsumm = np.array([['df_namekey', 'col_name'
                                 , 'col_type', 'col_nan_count'
                                 , 'total_rows']])
    
    for df_key in df_dict.keys():
        # append each column's info to the array
        for col in df_dict[df_key].columns:
            arr_colsumm = np.append(arr_colsumm
               , [[df_key
                 , col
                 , df_dict[df_key][col].dtype
                 , df_dict[df_key][col].isna().sum()
                 , len(df_dict[df_key])]], axis=0)

    df_colsumm = pd.DataFrame(arr_colsumm[1:], columns=arr_colsumm[0])
    
    return df_colsumm

In [11]:
files = ['bom.movie_gross.csv.gz'
         ,'imdb.title.ratings.csv.gz'
         ,'imdb.name.basics.csv.gz'
         ,'rt.movie_info.tsv.gz'
         ,'imdb.title.akas.csv.gz'
         ,'rt.reviews.tsv.gz'
         ,'imdb.title.basics.csv.gz'
         ,'tmdb.movies.csv.gz'
         ,'imdb.title.crew.csv.gz'
         ,'tn.movie_budgets.csv.gz'
         ,'imdb.title.principals.csv.gz']

df_names = ['df_bom_gross'
         ,'df_imdb_ratings'
         ,'df_imdb_namebasics'
         ,'df_rt_movie_info'
         ,'df_imdb_akas'
         ,'df_rt_reviews'
         ,'df_imdb_basics'
         ,'df_tmdb_movies'
         ,'df_imdb_crew'
         ,'df_tn_budgets'
         ,'df_imdb_principals']

### Initial preview of first 10 rows of each file

First, I loaded only the first 10 rows of each file into a dataframe and displayed the `head()`. This function also counts the total lines in each file.

I reviewed the results from each file to understand what information they contain. In terms of what I'm hoping for, I'm not sure whether I have sufficient data to calculate the actual profit. Ideally, I would like the profit calculation to be something like:

sum(box office gross, streaming revenue, VOD revenue, merchandising revenue)

minus


sum(production actual costs, advertising spend, theaters' cut)

In [12]:
#[preview_csvtsv(file, 10) for file in files]

### Import files into data frames and summarize columns

Next, I ran a function that attempts to actually import the full files into data frames. It returns a dictionary with the keys being the df names I specified, and the values being the dataframes themselves.

One of the 'tsv' files from Rotten Tomatoes encountered errors while importing, although there were no errors with the initial 10-row preview. I was able to run through another function that has more error handling, and adjust the encoding to get a good import.

Finally, I ran the dictionary of dataframes through a function which summarizes the columns. I think this will help me see which columns I have.

In [13]:
df_dict = filelist_to_dfs_prelim(files, df_names)
df_dict.keys()

Working on bom.movie_gross.csv.gz...
Working on imdb.title.ratings.csv.gz...
Working on imdb.name.basics.csv.gz...
Working on rt.movie_info.tsv.gz...
Working on imdb.title.akas.csv.gz...
Working on rt.reviews.tsv.gz...
Error encountered and rt.reviews.tsv.gz was skipped
Working on imdb.title.basics.csv.gz...
Working on tmdb.movies.csv.gz...
Working on imdb.title.crew.csv.gz...
Working on tn.movie_budgets.csv.gz...
Working on imdb.title.principals.csv.gz...


dict_keys(['df_bom_gross', 'df_imdb_ratings', 'df_imdb_namebasics', 'df_rt_movie_info', 'df_imdb_akas', 'df_imdb_basics', 'df_tmdb_movies', 'df_imdb_crew', 'df_tn_budgets', 'df_imdb_principals'])

In [14]:
df_dict2 = filelist_to_dfs_errorh(['rt.reviews.tsv.gz'], ['df_rt_reviews'])
df_dict2.keys()

Working on rt.reviews.tsv.gz...


dict_keys(['df_rt_reviews'])

In [15]:
df_dict2['df_rt_reviews']

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [16]:
# Merge dict of clean dfs with the dict of dfs that had errors
df_dict.update(df_dict2)
df_dict.keys()

dict_keys(['df_bom_gross', 'df_imdb_ratings', 'df_imdb_namebasics', 'df_rt_movie_info', 'df_imdb_akas', 'df_imdb_basics', 'df_tmdb_movies', 'df_imdb_crew', 'df_tn_budgets', 'df_imdb_principals', 'df_rt_reviews'])

### Create a summary of the columns in each table

Export this summary to a CSV file so I can make outside notes.

In [17]:
df_colsumm = summarize_cols(df_dict)
#df_colsumm.to_csv('column_summary.csv',index=False)

# SQL Ingestion for deeper analysis

I will create a SQLlite database and import the files I intend to use in my analysis into tables. I think this will be the best way to clean the data and save my changes outside of the Jupyter Notebook without a lot of file import/export.

I will be making notes for what I want to transform, date ranges, etc for each table and column outside of this notebook, and then will find a good way to get the information back in. This mostly saves me from scrolling up and down and makes it easier to take notes without worrying about formatting.

## Create New SQLLite database

In [23]:
cd ..

/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project


In [24]:
cd analysis

/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/analysis


In [26]:
# Create new database (or conenct to it, later)
conn = sqlite3.connect('movie_analysis.db')
cur = conn.cursor()

In [30]:
# Starting with the The Numbers data set, as profit is first analysis
# I want to tackle

# Get the column info
df_dict['df_tn_budgets'].head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Wait is this data any good??

Trying to determine whether the budget will always be in US dollars, whether it is adjusted for inflation, et. Trying to confirm whether domestic gross and worldwide gross include more than box office?

Interstingly, when I look up Avatar on The Numbers, the figures don't match what I see here.
https://www.the-numbers.com/movie/Avatar#tab=summary

Gross numbers more or less match what I see under box office, so I think those are box office only.

But production budget is only 237 compared to $425 MM that I see in my table.

I tried their movie comparison tool which lets you compare movies and adjust for inflation, and I'm not sure where the $425 MM is coming from, It's not just inflation. With inflation adjustment Avatar's budget is still only 289 million. https://www.the-numbers.com/movies/custom-comparisons/Avatar/Titanic-(1997)?mode=actuals#tab=day_by_day_comparison

Looking deeper at source which have APIs, The Movie Database has a free API with NO rate limiting.

I spot checked Avatar and Titanic across The Numbers and the revenue and budget are roughly equivalent (The Numbers had slightly higher revenue so may be more up to date than TMDb but pretty close.

Going to try getting the data from TMDB. Bonus is that my Watchmode data has TMDB ID, so I can easily join them.

# Initial Analysis Notes and thoughts:
The question I am asked to investigate is: "which movies are doing best at the box office?" But if the business goal is successfully entering the movie-making market, this question is only one of many we should ask and answer to determine the best approach.

Considering only the past earnings from box office sales is unlikely to be a good predictor of successful movies in the future. Even if we were to use only the past few years of data, the business model is rapidly changing. With the rise of streaming, many movies aren't released in theaters at all anymore, and the future of movie theaters has become especially uncertain due to the Covid-19 pandemic. 

The attributes of movies that are the most successful in the theaters may vary from the attributes of movies that are most successful when released only on streaming platforms. The reasons why people may choose to go and watch a movie in the theaters will be different from the reasons people choose to stream a movie: theaters offer an event-like experience with the very large screen and loud surround sound, whereas viewers streaming movies watch them in their homes. 

It's unclear whether the entire business model of creating movies for streaming services differs from the traditional model. 

## Look deeper into what we have in each data set

Answer questions such as:
- What information is in each data set? Does any single data set have everything you need, or do you need to figure out how to join data sets?
- What is the range of release years in each data set? Which have the most data?
- What data points do I want to focus on in each data set, and what can I discard/disregard? This will narrow down the data I intend to clean up.

1. Monetary stuff in The Numbers data set - box office gross and budget. Look at changes over time to attempt to identify trends.
    - Is there a sweet spot in terms of a  budget range to maximize profit? Are there diminishing returns?
    - Do we see any trends in profits over time? Has the distribution of profits coming from domestic versus foreign changed over time?
2. What are the attributes of the most profitable movies? Do they get higher critical ratings? Is there a certain sweet spot for runtime length? How about MPAA ratings? Genres? Release month?
    - Should the studio focus on making movies that make critics happy to maximize profits?
3. What can we learn about the contributors to movies? If we look at the writers, directors, producers on the most profitable movies, then look at other movies they were in, do we see any patterns? How about the actors and actresses? Do more experienced contributors (those who have worked on x amount of movies prior) create a more profitable product?
    - Should Microsoft focus on hiring certain types of talent? Should they consider paying a premium for more famous actors or directors?


## Measures of successful movies include:

|Measure|Have it?|Where|Notes|
|---|---|---|---|
|Gross earnings (box office only)|Yes|bom.movie_gross.csv.gz, tn.movie_budgets.csv.gz, rt_movie_info|BOM splits into foreign and domestic. TN has the most records, AND prod budget. RT has only 1500.|
|Net box office earnings after subtracting costs|Some|Can calculate from gross - production budget|Other data sets have gross box office earnings but not production budget|df_tn_budget|
|Merchandising earnings|No|---|---|
|Gross earnings from other sources, such as streaming, rentals|No|---|
|High ratings from professional movie reviewers|Yes|rt.reviews.tsv.gz|RT is the only data set with critical reviews|
|High ratings from viewers|Yes|imdb.title.ratings.csv.gz, tmdb.movies.csv.gz|I think the tmdb ratings are users not critics|
|Being nominated for and winning awards|No|---|
|---|---|---|
 
Microsoft will need to decide which measures of success are most important. Given its primary business is profitable and in a totally different economic sector, it may decide to treat the foray into film as a way to keep the brand feeling "cool" to consumers. If that's the case, measures of success such as ratings and award nominations may be weighted more heavily than profit.

## Potential Independent Variables
Reasons why certain movies may be more successful than others (independent variables) include:

|Independent Variable|Have it?|Where|Notes|
|---|---|---|---|
|Budget - i.e. how much money they spent making it|Yes|tn.movie_budgets.csv.gz|
|Genre (action, drama, comedy, etc)|Yes|imdb.title.basics.csv.gz|
|Well-known and loved actors being in them|Yes, ish|imdb.name.basics.csv.gz, imdb.title.principals.csv.gz|Join tables to figure out who did what|
|Well-known and loved writers, directors, and other creators|Yes|imdb.name.basics.csv.gz,imdb.title.crew.csv.gz||
|Seasonality (combination of topic and release date)|No|---|
|Whether the movie is part of a series or franchise|Can get|---|Perhaps if based on a book or comic book? https://www.the-numbers.com/movies/franchises|
|Month of release|Yes|tmdb.movies.csv.gz|
|Method of release (in theaters only, straight to streaming, etc)|No|---|
|MPAA Rating|Yes|rt.movie_info.tsv.gz|
|Runtime length|Yes|rt.movie_info.tsv.gz, imdb.title.basics.csv.gz|
|Semantic characteristics of the title|Yes|imdb.title.akas.csv.gz|---|
|Semantic characteristics of the official synopsis|Yes|rt.movie_info.tsv.gz|---|



## Potential Trends to Consider
- Has there been any change in the popularity of movies directed by different categories of people? For instance, are movies written or directed by women becoming more popular over time?
- How is the method of release changing over time?
- Are budgets changing over time? Is the change in line with inflation, or changing at a higher rate?
- Is the general profitability changing over time? Should Microsoft rethink getting into movies if the profit margins are getting slimmer and slimmer?
- How many very successful movies are made by smaller, independent studios? How long have the most successful studios been established?
- Is there a really big gap between the highest earners / most popular and the average? Do the movies we consider successful stand way out from the pack? What's the standard deviation?

## Sources used for research
- In general, major studios don’t disclose the full budgets for their films (production, development, marketing, and advertising).[1]
- In addition to the cost of production, the gross box office earnings don't factor in that the theaters keep a cut of the ticket sales. It's hard to know what the production company actually gets.[1]
- The movie industry is rather famously difficult to predict accurately. Many have tried, but it's difficult to do. [2]
- One hit can pay for a lot of flops [2]
- Many of the existing production companies have been around for a long time [2]
-"Because it’s harder to get financing and audiences, companies are competing to make bigger, costlier films while eliminating risk, which is why ever-more movies are based on existing intellectual property. Eighteen of the all-time 100 top-grossing movies (adjusted for inflation) were sequels, and more than half of those were released since 2000." [2]


[1]: Source = https://www.investopedia.com/articles/investing/093015/how-exactly-do-movies-make-money.asp#:~:text=VOD%20revenues%20tend%20to%20dry,audiences%20away%20from%20traditional%20movies Accessed on 3/12/2021.

[2] Source = https://www.nytimes.com/2012/07/01/magazine/how-does-the-film-industry-actually-make-money.html Accessed 3/12/2021.

 